In [1]:
import numpy as np

In [2]:
CRC_polynomial =np.array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])
CRC_len=len(CRC_polynomial)

In [3]:
def left_cyclic(data,polynomial,memory):
  tmp=np.zeros(len(memory)+1,dtype=int)
  tmp[0]=data
  tmp[1:]=memory
  res=np.sum(tmp*polynomial)%2
  memory=tmp[:len(memory)]
  return res,memory

def cyclic(data,polynomial,memory):

  tmp=-1*np.ones(len(memory)+1,dtype=int)
  tmp[len(tmp)-1]=data
  pre_data=memory[0]
  tmp[:len(tmp)-1]=memory

  for i in range(len(polynomial)-1):
    if polynomial[i]==1:
      tmp[i]=(pre_data+tmp[i+1])%2
    else:
      tmp[i]=tmp[i+1]

  #from IPython.core.debugger import Pdb; Pdb().set_trace()
  memory=tmp[:len(memory)]

  return memory

In [4]:
CRC_polynomial =np.array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])
info_len=15
information=np.random.randint(0,2,info_len)
#print(information)

def CRC_gen(information,parity,polynomial):
  CRC_info=np.zeros(len(information)+len(parity),dtype='int')
  CRC_info[:len(information)]=information
  CRC_info[len(information):]=parity

  memory=np.zeros(CRC_len-1,dtype='int')
  CRC_info[:len(information)]=information
  for i in range(len(CRC_info)):
    memory=cyclic(CRC_info[i],polynomial,memory)
  #print(len(memory))
  CRC_info[info_len:]=memory

  return CRC_info,np.all(memory==0)

parity=np.zeros(len(CRC_polynomial)-1)
CRC_info,check=CRC_gen(information,parity,CRC_polynomial)
print(information)
print(CRC_info)
parity=CRC_info[len(information):]
information=CRC_info[:len(information)]
CRC_info,check=CRC_gen(information,parity,CRC_polynomial)
print(check)

[1 0 0 1 0 1 1 0 0 0 0 0 1 0 1]
[1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 1 1 0 0 1 0 0 0 0]
True


In [5]:
def CRC_check(self,information):
  memory=np.zeros(CRC_len-1)
  for i in range(len(CRC_info)):
    memory=self.right_cyclic(CRC_info[i],CRC_polynomial,memory)
  
  if np.all(memory==0):
    return True
  
  else:
    return False

In [ ]:

CRC_info=np.zeros(K+CRC_len-1)
memory=np.zeros(CRC_len-1,dtype=int)
for i in range(K):
  CRC_info[i],memory=left_cyclic(information[i],CRC_polynomial,memory)

CRC_info[self.K:]=memory

print(CRC_check(CRC_info))
if CRC_check(CRC_info)==False:
  print("CRC_err")

print(CRC_info)